In [1]:
import glob
import numpy as np
import pandas as pd
import requests
from io import BytesIO

In [ ]:
def parse_cords(cord_string):
    cords = [int(i) for i in cord_string.replace(']','').replace('[','').split(' ') if i]
    cords = np.reshape(cords, (len(cords)//4,4))
    return cords

def recal_box(box_cords, old_shape, new_shape):
    recal_boxes=[]
    
    for box in box_cords:
        Ry, Rx=new_shape[0]/old_shape[0],new_shape[1]/old_shape[1]

        x,y,w,h = box
        new_y, new_x = int(Ry*y), int(Rx*x)

        new_h, new_w = int(Ry*h), int(Rx*w)
        
        recal_boxes.append((new_x, new_y, new_w, new_h))

    return recal_boxes

def cropface(image, box, fill=.5, ratios=(1,1)):
    shape = image.shape
    if len(shape) > 2 :
        h_img,w_img,c_img = shape
    else:
        h_img,w_img = shape

    Ry, Rx = ratios
    x,y,w,h = box
    

    new_y,new_x = Ry*y, Rx*x
    y_fill = max(0, new_y-h*fill)
    x_fill = max(0, new_x-w*fill)

    new_h, new_w = Ry*(h+y), Rx*(w+x)
    
    h_fill = min(h_img, new_h+h*fill)
    w_fill = min(w_img, new_w+w*fill)
    
    return image[int(y_fill):int(h_fill),
               int(x_fill):int(w_fill)]


In [ ]:
csv_files=glob.glob('./face_box/*.csv')
len(csv_files)

In [ ]:
for i, csv_p in enumerate(csv_files):
    test_recal=pd.read_csv(csv_p)
    if len(parse_cords(test_recal['face_boxs'][0]))> 4:
        print(i)
        break

In [ ]:
test_recal=pd.read_csv(csv_files[192])
tsd=parse_cords(test_recal['face_boxs'][0])
tsd

In [ ]:
test_recal=pd.read_csv(csv_files[192])

image_bytes_s = requests.get(test_recal['photo_image_url'][0])

image_bytes_s = image_bytes_s.content
image_stream_s = BytesIO(image_bytes_s)
img_open_s = np.array(Image.open(image_stream_s))

image_bytes = requests.get(test_recal['photo_image_url'][0].split('?')[0])
image_bytes = image_bytes.content
image_stream = BytesIO(image_bytes)
img_open = np.array(Image.open(image_stream))

og_shape=img_open_s.shape
new_shape=img_open.shape

print(parse_cords(test_recal['face_boxs'][0]))
new_box=recal_box(parse_cords(test_recal['face_boxs'][0]),og_shape, new_shape)
print(new_box)
plt.imshow(cropface(img_open_s, parse_cords(test_recal['face_boxs'][0])[0]))

In [ ]:
plt.imshow(cropface(img_open, new_box[0]))

In [ ]:
# Going throught all the file and merging into one
csv_files = glob.glob('./face_box/*.csv')

result_dict = {
    'photo_image_url': [],
    'face_box_cords': []
}

for i, csv in enumerate(csv_files):
    
    cur = pd.read_csv(csv)
    try:
        image_bytes_s = requests.get(cur['photo_image_url'][0])

        image_bytes_s = image_bytes_s.content
        image_stream_s = BytesIO(image_bytes_s)
        img_open_s = np.array(Image.open(image_stream_s))

        image_bytes = requests.get(cur['photo_image_url'][0].split('?')[0])
        image_bytes = image_bytes.content
        image_stream = BytesIO(image_bytes)
        img_open = np.array(Image.open(image_stream))

        og_shape = img_open_s.shape
        new_shape = img_open.shape    

        result_dict['face_box_cords'].append(recal_box(parse_cords(cur['face_boxs'][0]),og_shape, new_shape))
        # result_dict['face_box_cords'].append(parse_cords(cur['face_boxs'][0]))
        result_dict['photo_image_url'].append(cur['photo_image_url'][0].split('?')[0])
    except:
        print(f'error with {csv}')
    print(f'{i}/{len(csv_files)}', end='\r')
    

In [ ]:
final_merge = pd.DataFrame.from_dict(result_dict)

In [ ]:
final_merge.head()

In [ ]:
final_merge.to_csv('./merge_face_cords.csv', sep=';', index=False)

In [ ]:
final_merge=pd.read_csv('./merge_face_cords.csv', sep=';')

In [ ]:
final_merge.head()

In [ ]:
final_merge['photo_image_url']=final_merge['photo_image_url'].str.split('?').str[0]

In [ ]:
final_merge['photo_image_url'].iloc[10]

In [ ]:
final_merge.to_csv('./merge_face_cords_full.csv', sep=';', index=False)